# Enrichment of regulons from Regulons_by_strain

In [45]:
import os
import sys
from importlib import reload

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)

import pandas as pd
from IPython.display import display
pd.set_option('max_colwidth', 70)
pd.set_option('display.max_rows', 500)
import string
import numpy as np

from orangecontrib.bioinformatics.geneset.__init__ import list_all

import jupyter_functions as jf
reload(jf)
import jupyter_functions as jf
from functionsDENet import loadPickle, savePickle
import stages_DE.enrichment_library as enr
reload(enr)
import stages_DE.enrichment_library as enr

In [46]:
#Path for expression data (mergedGenes_RPKUM.tsv)
dataPath='/home/karin/Documents/timeTrajectories/data/RPKUM/combined/'
#Path for regulons
pathRegulons='/home/karin/Documents/timeTrajectories/data/regulons/by_strain/kN300_mean0std1_log/'

## Used gene sets
Use gene sets that do not have less than 5 or more than 500 genes. Use all ontologies.

In [59]:
# Get all gene sets
gene_sets=list(list_all(organism='44689'))
GENE_SETS_ONTOLOGY=enr.get_gene_sets(gene_set_names=gene_sets, go_slims=True,set_sizes=(5,500))

In [60]:
# Find out max/min gene set sizes and N sets
max_n=0
min_n=np.inf
for k,v in GENE_SETS_ONTOLOGY.items():
    for s in v:
        n=len(s.genes)
        if n<min_n:
            min_n=n
        if n>max_n:
            max_n=n
print('Min size',min_n,'Max size',max_n)  
print('N sets',{k:len(v) for k,v in GENE_SETS_ONTOLOGY.items()})

Min size 5 Max size 440
N sets {('Dictybase', 'Phenotypes'): 209, ('GO', 'biological_process'): 52, ('GO', 'molecular_function'): 35, ('GO', 'cellular_component'): 24, ('KEGG', 'Pathways'): 96}


In [49]:
# For conversion of gene names to EID
genes = pd.read_csv(dataPath + 'mergedGenes_RPKUM.tsv', sep='\t', index_col=0)
all_gene_names= genes[(genes != 0).any(axis=1)].index
NAME_EID=enr.name_genes_entrez(gene_names=all_gene_names, key_entrez=False)
ALL_GENE_NAMES_EID=enr.convert_EID(genes=all_gene_names, name_EID=NAME_EID)

In [64]:
def group_diff_enrichment(data:pd.DataFrame,group:str,padj:float=0.25,min_overlap:int=None,
                          use_annotated_genes:bool=False):
    #Displays only gene sets that have overlap with query greater or equal to min_overlap 
    #For p value and padj calculation uses alll that have overlap >=1 } from gene_set_enrichment
    """
    :param use_annotated_genes: if True use for reference and query  only genes that have at 
    least one gene set annotation
    """
    selected=list(data[data['Cluster']==group].index)
    query_EID=enr.convert_EID(genes=selected, name_EID=NAME_EID)
    print('***  '+group+' selected:',len(selected),'with EID:',len(query_EID))
    
    reference_gene_eids=ALL_GENE_NAMES_EID.copy()
    query_eids=query_EID.copy()
    
    if use_annotated_genes:
        gene_sets_genes=set()
        for gene_set_name, gene_sets in GENE_SETS_ONTOLOGY.items():
            for gene_set in gene_sets:
                gene_sets_genes.update(gene_set.genes)
        reference_gene_eids=set(reference_gene_eids) & gene_sets_genes
        query_eids=set(query_eids) & gene_sets_genes
        print('Ratio of genes annotated with a gene set in reference',
              round(len(reference_gene_eids)/len(ALL_GENE_NAMES_EID),2),
             'and query',round(len(query_eids)/len(query_EID),2))
    
    query_in_enriched=set()
    result=None
    if len(query_eids) > 0:
        enrichment=enr.gene_set_enrichment(query_eids, reference_EID=reference_gene_eids, 
                                                padj_threshold=padj,min_overlap=min_overlap,
                                                gene_sets_ontology=GENE_SETS_ONTOLOGY)
        if len(enrichment)>0:
            enrichment_display=list()
            enrichment= sorted(enrichment, key=lambda data: data.padj)
            for enriched in enrichment:
                query_in_enriched.update(enriched.gene_set.genes & query_eids)
                enrichment_display.append({'Gene set':enriched.gene_set.name,
                                           'Ontology':enriched.ontology[0]+': '+enriched.ontology[1],
                                           'FDR':"{:.2e}".format(enriched.padj),'N in query':enriched.in_query,
                                           'Set size':len(enriched.gene_set.genes),
                                                          'N in ref.':enriched.in_reference})
            result=pd.DataFrame(enrichment_display)
    print('Enrichment at FDR: '+str(padj)+' and min query - gene set overlap',str(min_overlap))
    print('N query genes in displayed gene sets:',len(query_in_enriched),'out of', len(query_eids),
          'query genes used for enrichment calculation.')
    display(result)
    print('\n')
    return result

## Enrichment reports

In [41]:
save_enrichment=False

### Enrichment for expression_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.pdf

In [65]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 30 with EID: 30
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 30 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Ribosome biogenesis in eukaryotes,KEGG: Pathways,3.61e-19,12,72,68
1,nucleolus,GO: cellular_component,4.81e-17,11,72,72
2,ribosome biogenesis,GO: biological_process,5.97e-16,10,61,61
3,ribonucleoprotein complex assembly,GO: biological_process,3.39e-07,4,18,18
4,RNA binding,GO: molecular_function,8.74e-06,6,169,169
5,protein-containing complex assembly,GO: biological_process,7.59e-04,4,134,133
6,rRNA binding,GO: molecular_function,1.00e-03,2,13,13
7,cellular component assembly,GO: biological_process,2.69e-03,4,202,201
8,helicase activity,GO: molecular_function,1.20e-02,2,51,51
9,methyltransferase activity,GO: molecular_function,1.30e-02,2,58,56




***  C2 selected: 66 with EID: 65
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 65 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Oxidative phosphorylation,KEGG: Pathways,6.60e-10,9,71,54
1,mitochondrion,GO: cellular_component,1.23e-04,9,229,229
2,Citrate cycle (TCA cycle),KEGG: Pathways,4.91e-04,4,30,30
3,Phagosome,KEGG: Pathways,3.34e-03,4,53,52
4,protein folding,GO: biological_process,1.16e-02,3,37,37
5,generation of precursor metabolites and energy,GO: biological_process,1.16e-02,4,77,77
6,cytoplasmic vesicle,GO: cellular_component,1.16e-02,6,211,211
7,aberrant cellular response to stress,Dictybase: Phenotypes,1.55e-02,2,11,11
8,Ribosome,KEGG: Pathways,1.59e-02,4,105,97
9,Biosynthesis of secondary metabolites,KEGG: Pathways,3.01e-02,6,289,281




***  C3 selected: 68 with EID: 68
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 62 out of 68 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,cell adhesion,GO: biological_process,8.68e-02,3,84,83
1,extracellular region,GO: cellular_component,8.68e-02,4,157,157
2,hydrolase activity,GO: molecular_function,1.57e-01,2,46,45
3,cell differentiation,GO: biological_process,1.92e-01,3,145,145
4,anatomical structure formation involved in morphogenesis,GO: biological_process,1.96e-01,2,74,74




***  C20 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 4 out of 54 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,decreased spore viability,Dictybase: Phenotypes,2.34e-01,2,60,59
1,Phagosome,KEGG: Pathways,2.34e-01,2,53,52




***  C21 selected: 45 with EID: 45
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 45 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set 
To find genes that have gene set annotations only gene sets used in enrichment are used (see above). Genes that are in at least one of these gene sets are used for enrichment calculation, both for reference and gene group of interest (query).

In [66]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2,use_annotated_genes=True)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_annotatedOnly_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 30 with EID: 30
Ratio of genes annotated with a gene set in reference 0.32 and query 0.7
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 21 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Ribosome biogenesis in eukaryotes,KEGG: Pathways,1.09e-15,12,72,68
1,nucleolus,GO: cellular_component,8.66e-14,11,72,72
2,ribosome biogenesis,GO: biological_process,6.22e-13,10,61,61
3,ribonucleoprotein complex assembly,GO: biological_process,7.03e-06,4,18,18
4,RNA binding,GO: molecular_function,5.86e-04,6,169,169
5,rRNA binding,GO: molecular_function,5.46e-03,2,13,13
6,protein-containing complex assembly,GO: biological_process,1.10e-02,4,134,133
7,cellular component assembly,GO: biological_process,4.00e-02,4,202,201
8,helicase activity,GO: molecular_function,5.27e-02,2,51,51
9,methyltransferase activity,GO: molecular_function,5.64e-02,2,58,56




***  C2 selected: 66 with EID: 65
Ratio of genes annotated with a gene set in reference 0.32 and query 0.58
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 33 out of 38 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Oxidative phosphorylation,KEGG: Pathways,8.92e-08,9,71,54
1,mitochondrion,GO: cellular_component,6.90e-03,9,229,229
2,Citrate cycle (TCA cycle),KEGG: Pathways,6.90e-03,4,30,30
3,Phagosome,KEGG: Pathways,3.07e-02,4,53,52
4,aberrant cellular response to stress,Dictybase: Phenotypes,7.32e-02,2,11,11
5,protein folding,GO: biological_process,7.32e-02,3,37,37
6,generation of precursor metabolites and energy,GO: biological_process,7.37e-02,4,77,77
7,cytoplasmic vesicle,GO: cellular_component,1.37e-01,6,211,211
8,Ribosome,KEGG: Pathways,1.37e-01,4,105,97
9,cell morphogenesis,GO: biological_process,1.57e-01,2,23,23




***  C3 selected: 68 with EID: 68
Ratio of genes annotated with a gene set in reference 0.32 and query 0.97
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 62 out of 66 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,cell adhesion,GO: biological_process,2.18e-02,3,84,83
1,extracellular region,GO: cellular_component,2.18e-02,4,157,157
2,cell differentiation,GO: biological_process,5.29e-02,3,145,145
3,hydrolase activity,GO: molecular_function,5.29e-02,2,46,45
4,anatomical structure formation involved in morphogenesis,GO: biological_process,1.02e-01,2,74,74




***  C20 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.2
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 6 out of 11 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,decreased spore viability,Dictybase: Phenotypes,1.51e-01,2,60,59
1,Phagosome,KEGG: Pathways,1.51e-01,2,53,52
2,extracellular region,GO: cellular_component,2.37e-01,2,157,157




***  C21 selected: 45 with EID: 45
Ratio of genes annotated with a gene set in reference 0.32 and query 0.13
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 3 out of 6 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,response to stress,GO: biological_process,2.25e-01,2,360,358
1,cytosol,GO: cellular_component,2.25e-01,2,203,203


### Enrichment for mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab

In [67]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    cluster='C'+str(group)
    print('*** Cluster %s (%s)' %(cluster,string.ascii_uppercase[group-1]))
    
    enriched=group_diff_enrichment(data=regulons,group=cluster,min_overlap=2)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

*** Cluster C1 (A)
***  C1 selected: 31 with EID: 31
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 31 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Ribosome biogenesis in eukaryotes,KEGG: Pathways,7.70e-19,12,72,68
1,nucleolus,GO: cellular_component,9.74e-17,11,72,72
2,ribosome biogenesis,GO: biological_process,1.15e-15,10,61,61
3,ribonucleoprotein complex assembly,GO: biological_process,5.11e-07,4,18,18
4,RNA binding,GO: molecular_function,1.41e-05,6,169,169
5,protein-containing complex assembly,GO: biological_process,1.13e-03,4,134,133
6,rRNA binding,GO: molecular_function,1.41e-03,2,13,13
7,cellular component assembly,GO: biological_process,4.00e-03,4,202,201
8,helicase activity,GO: molecular_function,1.68e-02,2,51,51
9,methyltransferase activity,GO: molecular_function,1.82e-02,2,58,56




*** Cluster C2 (B)
***  C2 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 25 out of 54 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,mitochondrion,GO: cellular_component,7.89e-08,11,229,229
1,Oxidative phosphorylation,KEGG: Pathways,7.89e-08,7,71,54
2,generation of precursor metabolites and energy,GO: biological_process,1.55e-05,6,77,77
3,protein folding,GO: biological_process,2.28e-04,4,37,37
4,unfolded protein binding,GO: molecular_function,2.02e-03,3,27,27
5,oxidoreductase activity,GO: molecular_function,2.14e-03,6,210,207
6,Phagosome,KEGG: Pathways,1.01e-02,3,53,52
7,Propanoate metabolism,KEGG: Pathways,4.11e-02,2,32,32
8,Citrate cycle (TCA cycle),KEGG: Pathways,4.11e-02,2,30,30
9,Ribosome,KEGG: Pathways,4.11e-02,3,105,97




*** Cluster C3 (C)
***  C3 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 44 out of 54 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,external encapsulating structure,GO: cellular_component,1.83e-02,2,13,13
1,cell wall,GO: cellular_component,1.83e-02,2,13,13
2,anatomical structure formation involved in morphogenesis,GO: biological_process,4.02e-02,3,74,74
3,hydrolase activity,GO: molecular_function,1.05e-01,2,46,45
4,cell differentiation,GO: biological_process,1.20e-01,3,145,145
5,lipid binding,GO: molecular_function,1.20e-01,2,59,59
6,extracellular region,GO: cellular_component,1.25e-01,3,157,157




*** Cluster C19 (S)
***  C19 selected: 89 with EID: 88
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 88 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,extracellular region,GO: cellular_component,2.40e-03,7,157,157
1,anatomical structure formation involved in morphogenesis,GO: biological_process,1.81e-02,4,74,74
2,cell differentiation,GO: biological_process,9.71e-02,4,145,145
3,Starch and sucrose metabolism,KEGG: Pathways,9.71e-02,2,31,31




*** Cluster C20 (T)
***  C20 selected: 1 with EID: 1
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 1 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set

In [68]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    cluster='C'+str(group)
    print('*** Cluster %s (%s)' %(cluster,string.ascii_uppercase[group-1]))

    enriched=group_diff_enrichment(data=regulons,group=cluster,min_overlap=2,use_annotated_genes=True)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab'+str(group)+'.tsv', sep='\t',index=False)

*** Cluster C1 (A)
***  C1 selected: 31 with EID: 31
Ratio of genes annotated with a gene set in reference 0.32 and query 0.71
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 22 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Ribosome biogenesis in eukaryotes,KEGG: Pathways,3.11e-15,12,72,68
1,nucleolus,GO: cellular_component,2.24e-13,11,72,72
2,ribosome biogenesis,GO: biological_process,1.48e-12,10,61,61
3,ribonucleoprotein complex assembly,GO: biological_process,1.12e-05,4,18,18
4,RNA binding,GO: molecular_function,1.02e-03,6,169,169
5,rRNA binding,GO: molecular_function,7.87e-03,2,13,13
6,protein-containing complex assembly,GO: biological_process,1.71e-02,4,134,133
7,cellular component assembly,GO: biological_process,6.17e-02,4,202,201
8,helicase activity,GO: molecular_function,7.54e-02,2,51,51
9,methyltransferase activity,GO: molecular_function,8.06e-02,2,58,56




*** Cluster C2 (B)
***  C2 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.57
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 24 out of 31 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,Oxidative phosphorylation,KEGG: Pathways,5.95e-06,7,71,54
1,mitochondrion,GO: cellular_component,1.46e-05,11,229,229
2,generation of precursor metabolites and energy,GO: biological_process,3.63e-04,6,77,77
3,protein folding,GO: biological_process,2.05e-03,4,37,37
4,unfolded protein binding,GO: molecular_function,1.08e-02,3,27,27
5,oxidoreductase activity,GO: molecular_function,3.78e-02,6,210,207
6,Phagosome,KEGG: Pathways,5.08e-02,3,53,52
7,Citrate cycle (TCA cycle),KEGG: Pathways,1.34e-01,2,30,30
8,Propanoate metabolism,KEGG: Pathways,1.35e-01,2,32,32
9,Ribosome,KEGG: Pathways,1.87e-01,3,105,97




*** Cluster C3 (C)
***  C3 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.89
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 44 out of 48 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,anatomical structure formation involved in morphogenesis,GO: biological_process,3.14e-03,3,74,74
1,external encapsulating structure,GO: cellular_component,3.14e-03,2,13,13
2,cell wall,GO: cellular_component,3.14e-03,2,13,13
3,cell differentiation,GO: biological_process,1.39e-02,3,145,145
4,hydrolase activity,GO: molecular_function,1.39e-02,2,46,45
5,extracellular region,GO: cellular_component,1.39e-02,3,157,157
6,lipid binding,GO: molecular_function,2.02e-02,2,59,59
7,anatomical structure development,GO: biological_process,2.88e-02,4,440,438
8,protein transport,GO: biological_process,1.23e-01,2,197,197




*** Cluster C19 (S)
***  C19 selected: 89 with EID: 88
Ratio of genes annotated with a gene set in reference 0.32 and query 0.2
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 18 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query,Set size,N in ref.
0,extracellular region,GO: cellular_component,5.93e-05,7,157,157
1,anatomical structure formation involved in morphogenesis,GO: biological_process,2.76e-03,4,74,74
2,cell differentiation,GO: biological_process,2.30e-02,4,145,145
3,Starch and sucrose metabolism,KEGG: Pathways,4.05e-02,2,31,31




*** Cluster C20 (T)
***  C20 selected: 1 with EID: 1
Ratio of genes annotated with a gene set in reference 0.32 and query 0.0
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 0 query genes used for enrichment calculation.


None